# 50.039 Theory and Practice of Deep Learning Project 2024

Group 10
- Issac Jose Ignatius (1004999)
- Mahima Sharma (1006106)
- Dian Maisara (1006377)


## Motivation

Chest radiography is an essential diagnostic tool used in medical imaging to visualise structures and organs within the chest cavity. It is crucial for diagnosing various respiratory and heart-related conditions. However, with the increased demand for radiological reports within shorter timeframes to detect and treat illnesses, there have been insufficient radiologists available to perform such tasks at scale. Therefore, automated chest radiograph interpretation could provide substantial benefits supporting large-scale screening and population health initiatives. Deep-learning algorithms can be used to bridge this gap. They have been used for image classification, anomaly detection, organ segmentation, and disease progression prediction.
<br><br>

*In this project, we aim to train a deep neural network to perform multi-label image classification on a wide array of chest radiograph images that exhibit various pathologies.*<br><br>



---




## Import all relevant libraries

In [1]:
# Numpy
import numpy as np
# Pandas
import pandas as pd

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, RandomSampler
import torchvision
import torchvision.transforms as T
from torchvision.transforms import v2
from torchvision.io import read_image, ImageReadMode
from torchmetrics.classification import MulticlassAccuracy


# File Operations
import os

# Helper scripts
from tqdm.notebook import tqdm, tnrange
import math
# import sys
# sys.path.insert(0, '../src')
# from saver_loader import *
# %reload_ext autoreload
# %autoreload 2

print(torchvision.__version__)

0.17.2+cu121


In [2]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu") 
print(device)

cuda


## Data Loading

The training and validation datasets are from the **CheXphoto dataset** (Philips et al., 2020). <br><br> CheXphoto comprises a training set of natural photos and synthetic transformations of 10,507 X-ray images from 3,000 unique patients (32,521 data points) sampled at random from the CheXpert training dataset and an accompanying validation set of natural and synthetic transformations applied to all 234 X-ray images from 200 patients with an additional 200 cell phone photos of x-ray films from another 200 unique patients (952 data points).

### Retrieving dataset from Google Cloud Storage (GCS)




In [3]:
# # Connect to GCS to access data
# from google.colab import auth
# auth.authenticate_user() # TODO: everyone to send me gmail so I can have you authed for bucket access

# project_id = 'tpdl-414711'
# bucket_name = 'chexphoto-v1'
# !gcloud config set project {project_id}

# # Install Cloud Storage FUSE.
# !echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
# !apt -qq update && apt -qq install gcsfuse

# # Mount a Cloud Storage bucket or location, without the gs:// prefix.
# mount_path = "chexphoto-v1"  # or a location like "my-bucket/path/to/mount"
# local_path = f"/mnt/gs/{mount_path}"

# !mkdir -p {local_path}
# !gcsfuse --implicit-dirs {mount_path} {local_path}

### Setup environment variables


In [4]:
data_path = os.path.join(os.path.abspath(''), "../ChexPhoto/chexphoto-v1")
print(data_path)

c:\Users\User\Desktop\50.039 TPDL\2024_TPDL\notebooks\../ChexPhoto/chexphoto-v1


### Loading dataset (image and labels)

In [5]:
# Bug in Path present in training dataset
def fix_error_paths(row):
    row = row.replace("//", "/")
    return row

def str_to_array(row):
    ndarray = np.fromstring(
                row.replace('\n','')
                    .replace('[','')
                    .replace(']','')
                    .replace('  ',' '), 
                    sep=' ')
    return ndarray

In [6]:
train_df = pd.read_csv("../data/processed/train_one_hot_encoded.csv", index_col=False)
labels = train_df.columns[1:]
print(len(labels), labels)

13 Index(['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')


In [7]:
train_df["Path"] = train_df["Path"].apply(fix_error_paths)

valid_df = pd.read_csv("../data/processed/valid_one_hot_encoded.csv", index_col=False)
#test_df = pd.read_csv("../data/processed/test_one_hot_encoded.csv", index_col=False)

for label in labels:
    train_df[label] = train_df[label].apply(str_to_array)
    valid_df[label] = valid_df[label].apply(str_to_array)
    #test_df[label] = test_df[label].apply(str_to_array)

display(train_df)
display(valid_df)
#display(test_df)

,Path,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
1,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
2,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
3,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
4,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32516,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
32517,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
32518,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
32519,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"


,Path,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
1,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
2,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
3,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
4,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
698,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
699,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
700,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"


### Subsetting dataset (Binary Classification for Pleural Effusion)


In [8]:
def keep_observations(df, cols):
    return df[cols].copy()

In [9]:
# Drop all other labels
cols = ["Path", "Pleural Effusion", "Cardiomegaly"]
df_train = keep_observations(train_df, cols)
df_valid = keep_observations(valid_df, cols)
# df_test = keep_observations(test_df, cols)

In [10]:
def ohe_to_class(row):
    if np.sum(row) > 0:
        return np.argmax(row)
    else:
        return -100

df_train["Class_pEff"] = df_train["Pleural Effusion"].apply(ohe_to_class)
df_train["Class_Cardio"] = df_train["Cardiomegaly"].apply(ohe_to_class)

df_valid["Class_pEff"] = df_valid["Pleural Effusion"].apply(ohe_to_class)
df_valid["Class_Cardio"] = df_valid["Cardiomegaly"].apply(ohe_to_class)

In [11]:
display(df_train)
display(df_valid)
# display(df_test)

,Path,Pleural Effusion,Cardiomegaly,Class_pEff,Class_Cardio
0,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100
1,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100
2,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",-100,-100
3,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",-100,-100
4,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]",2,2
...,...,...,...,...,...
32516,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32517,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32518,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32519,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100


,Path,Pleural Effusion,Cardiomegaly,Class_pEff,Class_Cardio
0,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]",1,2
1,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
2,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
3,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
4,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
...,...,...,...,...,...
697,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
698,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
699,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]",1,2
700,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1


In [12]:
def sum_array(row):
    return np.sum(row)

df_train["Pleural Effusion_sum"] = df_train["Pleural Effusion"].apply(sum_array)
df_train["Cardiomegaly_sum"] = df_train["Cardiomegaly"].apply(sum_array)

display(df_train)

,Path,Pleural Effusion,Cardiomegaly,Class_pEff,Class_Cardio,Pleural Effusion_sum,Cardiomegaly_sum
0,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100,1.0,0.0
1,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100,1.0,0.0
2,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",-100,-100,0.0,0.0
3,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",-100,-100,0.0,0.0
4,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]",2,2,1.0,1.0
...,...,...,...,...,...,...,...
32516,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100,1.0,0.0
32517,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100,1.0,0.0
32518,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100,1.0,0.0
32519,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100,1.0,0.0


In [13]:
df_train = df_train[(df_train["Pleural Effusion_sum"] > 0) | (df_train["Cardiomegaly_sum"] > 0)]

# filter out NaNs given that we feed class labels to CrossEntropyLoss
cols = ["Path", "Pleural Effusion", "Cardiomegaly", "Class_pEff", "Class_Cardio"]
df_train = df_train[cols]
display(df_train)

,Path,Pleural Effusion,Cardiomegaly,Class_pEff,Class_Cardio
0,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100
1,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100
4,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]",2,2
5,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
8,CheXphoto-v1.0/train/synthetic/digital/patient...,"[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0,-100
...,...,...,...,...,...
32516,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32517,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32518,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32519,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100


### Custom Dataset implementation

In [14]:
# Implementation of Custom Dataset Class for CheXPhoto Dataset
class CheXDataset(Dataset):
    # Accepts dataframe object and str
    def __init__(self, df: pd.DataFrame, px_size: int = 256):
        self.dataframe = df.copy()
        self.px_size = px_size

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x_path = data_path + "/" + self.dataframe.iloc[idx, 0].split("CheXphoto-v1.0", 1)[-1]
        
        transform = T.Compose([
            v2.Resize((self.px_size, self.px_size), interpolation=T.InterpolationMode.BICUBIC)
        ])

        resized_x_tensor = transform(read_image(x_path, mode = ImageReadMode.RGB)) /255

        # y = (torch.tensor(self.dataframe.iloc[idx, 1], dtype=torch.float32), torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.float32))

        y = (torch.tensor(self.dataframe.iloc[idx, 3]).type(torch.LongTensor), torch.tensor(self.dataframe.iloc[idx, 4]).type(torch.LongTensor))
        # y = torch.tensor(self.dataframe.iloc[idx, 3:]).type(torch.LongTensor)
        return resized_x_tensor, y

### Custom Dataloader

In [15]:
# Load into custom Dataset
train_data = CheXDataset(df_train)
valid_data = CheXDataset(df_valid)
# test_data = CheXDataset(df_test)

# Prepare random sampler for training subset
# train_sampler = RandomSampler(data_source=train_data, num_samples=int(0.1*len(pEff_train_data)))
# train_sampler = WeightedRandomSampler([1873/19582, 4976/19582, 12733/19582], int(len(pEff_train_data)))

# Load into DataLoader
batch_size = 128
# train_loader = DataLoader(train_data, batch_size, sampler=train_sampler)
train_loader = DataLoader(train_data, batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size)
# test_loader = DataLoader(test_data, batch_size)

In [16]:
x, (y1, y2) = train_data[0]
print(x, x.shape, x.dtype)
print(y1, y1.shape, y1.dtype)
print(y2, y2.shape, y2.dtype)

tensor([[[0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         ...,
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588]],

        [[0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         ...,
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588]],

        [[0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.

## Model Tuning

Our initial model is a simple feedforward neural network with multiple heads (2 heads) capable of classifying for both Cardiomegaly and Pleural Effusion. We will utilise the Cross-entropy loss function to optimise the model during training.

**This is a TODO since it can change**


### First iteration - Simple Convolutional Neural Network

#### Model

In [17]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int, pool_size: int = 2):
        super(ConvolutionBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()
        self.pooling = nn.MaxPool2d(pool_size, pool_size)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.activation(out)
        out = self.pooling(out)
        return out

In [18]:
class FCBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, dropout_rate: int=0.2):
        super(FCBlock, self).__init__()
        
        self.fc = nn.Linear(in_features, out_features, dtype=torch.float32)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        out = self.fc(x)
        out = self.activation(out)
        out = self.dropout(out)
        return out

In [19]:
class ClassifierBlock(nn.Module):
    def __init__(self, in_features: int, hidden_features: tuple[int], out_features: int, dropout_rate: int = 0.2):
        super(ClassifierBlock, self).__init__()

        self.inputs = FCBlock(in_features, hidden_features[0], dropout_rate)
        self.layers = nn.Sequential(*[FCBlock(hidden_features[i], hidden_features[i+1], dropout_rate) for i in range(len(hidden_features)-1)])
        self.linear = nn.Linear(hidden_features[-1], out_features, dtype=torch.float32)
    
    def forward(self, x):
        out = self.inputs(x)
        out = self.layers(out)
        out = self.linear(out)
        return out

In [20]:
class ConvolutionalNN(nn.Module):
    def __init__(self, channels: tuple[int], img_size: tuple[int], hidden_size: tuple[int], output_size: int, dropout_rate: int = 0.2):
        super(ConvolutionalNN, self).__init__()
        
        # Hyperparameters for Convolutional Layers
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.pool_size = 2

        # Input size of Image
        self.input_height, self.input_width = img_size

        # Calculate Input Size of Classifier
        for i in range(len(channels) - 1):
            self.input_height = math.floor((self.input_height + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            self.input_width =  math.floor((self.input_width  + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
        
        self.classifier_size = channels[-1] * self.input_height * self.input_width
        
        # Model Layers
        self.conv_layers = nn.Sequential(*[ConvolutionBlock(channels[i], channels[i+1], self.kernel_size, self.stride, self.padding, self.pool_size) for i in range(len(channels)-1)])
        self.classifier1 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)
        self.classifier2 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)


    def forward(self,x):
        # Convolutional Layers
        out1 = self.conv_layers(x)

        # keep batch size and flatten the rest
        out2 = out1.view(out1.size(0), -1)

        # Classifier Layers
        out3 = self.classifier1(out2)
        out4 = self.classifier2(out2)
        return out3, out4

In [21]:
# Create Model
channels = (3, 8, 16, 32, 64, 128, 256)
input_size = (256, 256)
hidden_size = (2048, 256) 
output_size = 3
dropout_rate = 0.2

model = ConvolutionalNN(channels, input_size, hidden_size, output_size, dropout_rate).to(device)
print(model)

ConvolutionalNN(
  (conv_layers): Sequential(
    (0): ConvolutionBlock(
      (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvolutionBlock(
      (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvolutionBlock(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode

#### Training

In [22]:
def train_loop(model, train_loader, optimizer, loss_pEff, loss_cardio, lambda1, lambda2):
    model.train()

    train_loss = 0
    train_total = 0

    train_accuracy_pEff = 0
    train_accuracy_cardio = 0

    accuracy_pEff = MulticlassAccuracy(num_classes=3, ignore_index=-100).to(device)
    accuracy_cardio = MulticlassAccuracy(num_classes=3, ignore_index=-100).to(device)

    for inputs, outputs in tqdm(train_loader):
        inputs_re, outputs_re_pEff, outputs_re_cardio = inputs.to(device), outputs[0].to(device), outputs[1].to(device)

        optimizer.zero_grad()
        preds = model(inputs_re)

        # Compute loss
        loss1 = loss_pEff(preds[0], outputs_re_pEff)
        loss2 = loss_cardio(preds[1], outputs_re_cardio)
        loss_value = lambda1*loss1 + lambda2*loss2
        
        # Backpropagation
        loss_value.backward()
        optimizer.step()

        # Compute metric
        train_loss += loss_value.item() * outputs[0].size(0)
        train_total += outputs[0].size(0)
        
        accuracy_pEff.update(preds[0], outputs_re_pEff)
        accuracy_cardio.update(preds[1], outputs_re_cardio)

    train_loss /= train_total
    train_accuracy_pEff = accuracy_pEff.compute() 
    train_accuracy_cardio = accuracy_cardio.compute()

    return train_loss, train_accuracy_pEff, train_accuracy_cardio

### TODO: Implementation Validation Loss

In [23]:
def test_loop(model, valid_loader, loss_pEff, loss_cardio, lambda1, lambda2):
    model.eval()
    
    val_loss = 0
    val_total = 0

    val_accuracy_pEff = 0
    val_accuracy_cardio = 0

    accuracy_pEff = MulticlassAccuracy(num_classes=3, ignore_index=-100).to(device)
    accuracy_cardio = MulticlassAccuracy(num_classes=3, ignore_index=-100).to(device)

    with torch.no_grad():
        for inputs, outputs in tqdm(valid_loader):
            outputs_pEff = outputs[0]
            outputs_cardio = outputs[1]
            inputs_re, outputs_re_pEff, outputs_re_cardio = inputs.to(device), outputs_pEff.to(device), outputs_cardio.to(device)
            preds = model(inputs_re)

            # Compute loss
            loss1 = loss_pEff(preds[0], outputs_re_pEff)
            loss2 = loss_cardio(preds[1], outputs_re_cardio)
            loss_value = lambda1*loss1 + lambda2*loss2

            # Compute metrics
            val_loss += loss_value.item() * outputs[0].size(0)
            val_total += outputs[0].size(0)

            accuracy_pEff.update(preds[0], outputs_re_pEff)
            accuracy_cardio.update(preds[1], outputs_re_cardio)

    val_loss /= val_total
    val_accuracy_pEff = accuracy_pEff.compute() 
    val_accuracy_cardio = accuracy_cardio.compute()

    return val_loss, val_accuracy_pEff, val_accuracy_cardio

In [24]:
def train(model, train_loader, valid_loader, epochs=10, lr=1e-3):
    # Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # loss hyperparameters - penalise minority class loss more
    # lambda1 = (19582+6812)/19582
    # lambda2 = (19582+6812)/6812

    lambda1 = 1
    lambda2 = 1

    # lambda1 = 1 # 19582/19582
    # lambda1 = 6812/19582
    # lambda2 = 19582/6812
    
    # class weights
    class_weights_pEff = torch.tensor([1873/19582, 4976/19582, 12733/19582]).to(device)
    class_weights_cardio = torch.tensor([1149/6812, 1623/6812, 4040/6812]).to(device)

    # loss function
    loss_pEff = nn.CrossEntropyLoss(weight=class_weights_pEff, ignore_index=-100).to(device)
    loss_cardio = nn.CrossEntropyLoss(weight=class_weights_cardio, ignore_index=-100).to(device)

    #train_loss_values = []
    #train_accuracy_values = []

    for epoch in tnrange(epochs):
        # Train loop
        train_loss, train_accuracy_pEff, train_accuracy_cardio = train_loop(model, train_loader, optimizer, loss_pEff, loss_cardio, lambda1, lambda2)

        # Test loop
        val_loss, val_accuracy_pEff, val_accuracy_cardio = test_loop(model, valid_loader, loss_pEff, loss_cardio, lambda1, lambda2)

        print(f"--- Epoch {epoch+1}/{epochs}: ---")
        print(f"Train loss: {train_loss:.4f}, Train accuracy (Pleural): {train_accuracy_pEff:.4f}, Train accuracy (Cardio): {train_accuracy_cardio:.4f}")
        print(f"Validation loss: {val_loss:.4f}, Validation accuracy (Pleural): {val_accuracy_pEff:.4f}, Validation accuracy (Cardio): {val_accuracy_cardio:.4f}")

In [25]:
epochs = 15
train(model, train_loader, valid_loader, epochs=epochs)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 1/15: ---
Train loss: 1.2041, Train accuracy (Pleural): 0.3338, Train accuracy (Cardio): 0.3338
Validation loss: 1.9324, Validation accuracy (Pleural): 0.5000, Validation accuracy (Cardio): 0.5000


  0%|          | 0/171 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 14/15: ---
Train loss: 0.5119, Train accuracy (Pleural): 0.5492, Train accuracy (Cardio): 0.6342
Validation loss: 2.4902, Validation accuracy (Pleural): 0.6917, Validation accuracy (Cardio): 0.4273


  0%|          | 0/171 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 15/15: ---
Train loss: 0.4658, Train accuracy (Pleural): 0.5629, Train accuracy (Cardio): 0.6588
Validation loss: 4.3670, Validation accuracy (Pleural): 0.6108, Validation accuracy (Cardio): 0.3844


### Second iteration - Convolutional neural network (CNN) with Skip Connection

#### Model

In [26]:
class SkipBlock(nn.Module):
    def __init__(self, in_channels: int, hidden_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int, pool_size: int = 2):
        super(SkipBlock, self).__init__()

        # Conv Layers + Skip Connections
        self.conv1 = ConvolutionBlock(in_channels, hidden_channels, kernel_size, stride, padding, pool_size)
        self.conv2 = ConvolutionBlock(hidden_channels, out_channels, kernel_size, stride, padding, pool_size)

        # TODO: Try out 2 different skip connection implementation
        # self.skip_connection = nn.Sequential(nn.Conv2d(channels[0], channels[2], kernel_size=1, stride=2*self.stride, padding=self.padding),
        #                                       nn.AvgPool2d(self.pool_size, self.pool_size))
        
        self.skip_connection = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=padding),
                                              nn.AvgPool2d(2*pool_size, 2*pool_size)
                                              )
        
    
    def forward(self, x):
        x1 = self.conv1(x)
        out = self.conv2(x1)

        skip = self.skip_connection(x)
        out += skip
        return out

In [27]:
# We will be using Convolutional Neural Network
class ConvolutionalNNv2(nn.Module):
    def __init__(self, channels: tuple[int], img_size: tuple[int], hidden_size: tuple[int], output_size: int, dropout_rate: int = 0.2):
        super(ConvolutionalNNv2, self).__init__()
        
        if len(channels) % 2 != 1:
            raise Exception("len(channels) must be an odd number!")
        elif channels[0] != 3:
            raise Exception("first element in channels must match the input channel size!")

        # Hyperparameters for Convolutional Layers
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.pool_size = 2

        # Input size of Image
        self.input_height, self.input_width = img_size

        # Calculate Input Size of Classifier
        for i in range(len(channels) - 1):
            self.input_height = math.floor((self.input_height + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            self.input_width =  math.floor((self.input_width  + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            
        self.classifier_size = channels[-1] * self.input_height * self.input_width


        # Conv Layers + Skip Connections
        self.skip_layers = nn.Sequential(*[SkipBlock(channels[i], channels[i+1], channels[i+2], self.kernel_size, self.stride, self.padding, self.pool_size) for i in range(0, len(channels)-2, 2)])

        # Dropout Layer
        self.dropout = nn.Dropout(dropout_rate)

        # Classifier Layers
        self.classifier1 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)
        self.classifier2 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)
        

    def forward(self,x):
        # SkipBlock layers
        x1 = self.skip_layers(x)

        # Flatten output of convolutions
        x2 = x1.view(x1.size(0), -1)
        # print(x.shape)
        x3 = self.dropout(x2)

        # Classifier layer
        x4 = self.classifier1(x3)
        x5 = self.classifier2(x3)
        
        return x4, x5

In [28]:
# Create Model
# model = torchvision.models.resnet18(weights=False).to(device) # to check if trainer works

channels = (3, 8, 16, 32, 64, 128, 256)
input_size = (256, 256)
hidden_size = (2048, 256) 
output_size = 3
dropout_rate = 0.2

model = ConvolutionalNNv2(channels, input_size, hidden_size, output_size, dropout_rate).to(device)
print(model)

ConvolutionalNNv2(
  (skip_layers): Sequential(
    (0): SkipBlock(
      (conv1): ConvolutionBlock(
        (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (conv2): ConvolutionBlock(
        (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (skip_connection): Sequential(
        (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
        (1): AvgPool2d(kernel_size=4, stride=4, padding=0)
      )
    )
    (1): SkipBlock(
      (conv1): ConvolutionBlock(
        (conv):

In [29]:
for inputs, outputs in train_loader:
    print(inputs.shape)
    print(outputs[0].shape, outputs[1].shape)
    break

torch.Size([128, 3, 256, 256])
torch.Size([128]) torch.Size([128])


In [30]:
epochs = 15
train(model, train_loader, valid_loader, epochs=epochs)
#torch.save(model.state_dict(), 'weights.pt')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 1/15: ---
Train loss: 1.0765, Train accuracy (Pleural): 0.3772, Train accuracy (Cardio): 0.3542
Validation loss: 1.9667, Validation accuracy (Pleural): 0.5359, Validation accuracy (Cardio): 0.5120


  0%|          | 0/171 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 14/15: ---
Train loss: 0.3582, Train accuracy (Pleural): 0.6034, Train accuracy (Cardio): 0.7973
Validation loss: 3.4419, Validation accuracy (Pleural): 0.4355, Validation accuracy (Cardio): 0.3859


  0%|          | 0/171 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 15/15: ---
Train loss: 0.3074, Train accuracy (Pleural): 0.6328, Train accuracy (Cardio): 0.8400
Validation loss: 3.0237, Validation accuracy (Pleural): 0.4357, Validation accuracy (Cardio): 0.3641


In [31]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.activation(out)
        return out

In [32]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2(x)
        out3 = self.conv3(x)
        out4 = self.conv4(x)
        out = torch.cat([out1, out2, out3, out4], 1)
        return out

### Testing

## Observations

**TODO** Discuss whether its right for us to pluck all our evaluation and training together and discuss it here or break up the code without any descriptions
